Already exists:
- Boligrafica Twitter Tweepy
- Greenhouse Summon
- NOAA Data update GHD

Want to create:
- GFW - Datos Bolivia Deforestacion / Incendios
- Other twitter bots!??

Types of conditions:
- Runs on a schedule 
  - once every week (like boligrafica)
  - 4 times a day (like greenhouse summon)
- Runs if it has detected updated data (if new data, run)
  - Like NOAA data in GHD

In [1]:
import os
import sys
import pandas as pd
import datetime
from IPython import get_ipython

# environmental secrets when working locally
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def NEW_DATA_RUNNER():
    
    return

In [3]:
def SCHEDULED_RUNNER(notebook_path, schedule_in_days = None):
    
    print("starting: ", notebook_path)
    
    if schedule_in_days == None:
        EXECUTE = True
        
    else:
        # check log
        log = pd.read_csv("log.csv", index_col=0)
        try:
            last_executed = log.loc[notebook_path]["last_executed"]
            print(f"previous log found: {last_executed}")
        except:
            print("no log found (first time?) - creating dummy entry (year 2000)")
            log.loc[notebook_path] = "2002-07-16 18:32"
            
            # try again
            last_executed = log.loc[notebook_path]["last_executed"]
            
        # get time difference in days
        now = datetime.datetime.now()
        last_executed = datetime.datetime.strptime(last_executed, '%Y-%m-%d %H:%M')
        diff_seconds = now - last_executed
        diff_days = diff_seconds.total_seconds() / 60 /60 /24
        print(f"difference in execution is: {round(diff_days,4)} days")
        print(f"needed difference to execute: {round(schedule_in_days,4)} days")
        
        # FINAL CHECK
        if diff_days > schedule_in_days:
            EXECUTE = True
        else:
            EXECUTE = False
            

        
    print("Execute is: ", str(EXECUTE))
    if EXECUTE == True:

        
        # go to folder to run script locally
        folder_path = notebook_path.split("/")[:-1]
        os.chdir("/".join(folder_path))

        try:
            get_ipython().run_line_magic("run", notebook_path.split("/")[-1])
            status = "SUCCESS!"
            print(status)
        except:
            status = "FAILURE."
            print(status)

        # go back to root repository
        os.chdir("".join(["../"] * len(folder_path)))
        
        
        # log
        log.loc[notebook_path] = [datetime.datetime.now().strftime("%Y-%m-%d %H:%M"), status]
        log.to_csv("log.csv")
        print("New log entry successful!\n\n")
    else:
        print("Did not execute nor log\n\n")
    return

########################################################
#

SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/Twitter/GreenhouseSummon.ipynb", schedule_in_days= 6/24)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/Random_Fig_Daily/Random_Fig_Daily.ipynb", schedule_in_days= 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/NOAA_Updates/NOAA_Update.ipynb", schedule_in_days= 1)


starting:  Greenhouse_Data/Twitter/GreenhouseSummon.ipynb
previous log found: 2022-07-23 08:13
difference in execution is: 0.3334 days
needed difference to execute: 0.25 days
Execute is:  True
Iran, Islamic Republic of greenhouse gas emission data and figures 👇

If interested be sure to check out my website for many more datasets and figures! Thank you and take care! 🤖
Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Barbados"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Falkland Islands (Malvinas)"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Peru"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Tuvalu"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Sea

['country_data/USA_United States/figures/USA_relative_totals.png',
 'country_data/USA_United States/figures/USA_GCP_Country_Highlight.png',
 'country_data/USA_United States/figures/USA_GCP_1.png',
 'country_data/USA_United States/figures/USA_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [True] 



Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Turkey"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions Morocco"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True]
 > Searching for 20 tweets containing "greenhouse gas emissions South Korea"
 > found 14 tweets
 > days of invalidity: 20
 > Number of invalids: 116
 > 4 usernames filtered out
 > retweets removed: 10


['country_data/KOR_South Korea/figures/KOR_relative_totals.png',
 'country_data/KOR_South Korea/figures/KOR_GCP_Country_Highlight.png',
 'country_data/KOR_South Korea/figures/KOR_GCP_1.png',
 'country_data/KOR_South Korea/figures/KOR_Minx_top20_subsectors.png']

 > saved and selected 4 images

 > OH NO! NO TWEETS WERE FOUND NOR TWEETED
success list: [True, False] 



Starting success list [True, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Romania"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Angola"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Togo"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Saint Helena, Ascension and Tristan da Cunha"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Lesotho"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False]
 > Searching for 20 tweets containing "green

...

 > saved and selected 4 images

 > OH NO! NO TWEETS WERE FOUND NOR TWEETED
success list: [True, False, False] 



Starting success list [True, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Guyana"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Colombia"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Guam"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [True, False, False]
 > Searching for 20 tweets containing "greenhouse gas emissions Finland"
 > found 4 tweets
 > days of invalidity: 20
 > Number of invalids: 116
 > 0 usernames filtered out
 > retweets removed: 1


['country_data/FIN_Finland/figures/FIN_relative_totals.png',
 'country_data/FIN_Finland/figures/FIN_GCP_Country_Highlight.png',
 'country_data/FIN_Finland/figures/FIN_GCP_1.png',
 'country_data/FIN_Finland/figures/FIN_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [True, False, False, True] 





 FINISHED WHILE LOOP


 FINISHED WHILE LOOP
Madagascar


['country_data/MDG_Madagascar/figures/MDG_relative_totals.png',
 'country_data/MDG_Madagascar/figures/MDG_GCP_Country_Highlight.png',
 'country_data/MDG_Madagascar/figures/MDG_GCP_1.png',
 'country_data/MDG_Madagascar/figures/MDG_Minx_top20_subsectors.png']

 > saved and selected 4 images
country profile tweet SUCCESS!
SUCCESS!
New log entry successful!


starting:  Greenhouse_Data/Random_Fig_Daily/Random_Fig_Daily.ipynb
previous log found: 2022-07-22 23:11
difference in execution is: 0.7103 days
needed difference to execute: 1 days
Execute is:  False
Did not execute nor log


starting:  Greenhouse_Data/NOAA_Updates/NOAA_Update.ipynb
previous log found: 2022-07-22 23:11
difference in execution is: 0.7103 days
needed difference to execute: 1 days
Execute is:  False
Did not execute nor log


